## Usability analysis setup

Use this section to setup the functions needed to perform a usability analysis of the dataset, to identify if a publisher has the necessary fields to calculate 71 procurement indicators related to market opportunity (market description, competition, supplier performance), value for money, internal efficiency, public integrity and service delivery.  For an OCDS publisher, it also calculates the proportion of unique procedures for which it is possible to calculate the indicator (coverage).

The usability checks includes all the indicators listed on [OCP's use case guide](https://docs.google.com/spreadsheets/d/1j-Y0ktZiOyhZzi-2GSabBCnzx6fF5lv8h1KYwi_Q9GM/edit#gid=1183427361) and the [Indicators to diagnose the performance of a procurement market document](https://docs.google.com/document/d/1vSJk9-qWSTQEx9ZZc7BUhQZMHvTRcyDYVS2sl8HB__k/edit#heading=h.nrnq1ajwwpqe).

In [ ]:
# @title Usability functions { display-mode: "form" }

RELEVANT_RULES = {
    "who": [
        "buyer/id",
        "buyer/name",
        "tender/procuringEntity/id",
        "tender/procuringEntity/name",
    ],
    "bought what": [
        "tender/items/classification/id",
        "awards/items/classification/id",
        "contracts/items/classification/id",
        "tender/items/classification/description",
        "awards/items/classification/description",
        "contracts/items/classification/description",
        "tender/items/description",
        "awards/items/description",
        "contracts/items/description",
        "tender/description",
        "awards/description",
        "contracts/description",
        "tender/title",
        "awards/title",
        "contracts/title",
    ],
    "from whom": [
        "awards/suppliers/id",
        "awards/suppliers/name",
    ],
    "for how much": [
        "awards/value/amount",
        "contracts/value/amount",
        [
            "awards/items/quantity",
            "awards/items/unit/value/amount",
        ],
        [
            "contracts/items/quantity",
            "contracts/items/unit/value/amount",
        ],
    ],
    "when": [
        "tender/tenderPeriod/endDate",
        "awards/date",
        "contracts/dateSigned",
    ],
    "how": [
        "tender/procurementMethod",
        "tender/procurementMethodDetails",
    ],
}

_BUYER = {
    "any": [
        {"all": ["buyer/name", "buyer/id"]},
        {"all": ["tender/procuringEntity/name", "tender/procuringEntity/id"]},
        {"all": ["parties/identifier/name", "parties/identifier/id", "parties/roles"]},
    ]
}

_BIDDERS = {"any": ["tender/tenderers/id", "bids/details/tenderers/id"]}

_ITEMS = {
    "any": [
        {"all": ["tender/items/classification/id", "tender/items/classification/scheme"]},
        {"all": ["awards/items/classification/id", "awards/items/classification/scheme"]},
        {"all": ["contracts/items/classification/id", "contracts/items/classification/scheme"]},
    ]
}

_AWARD_CONTRACT_STATUS = {
    "any": [
        {"all": ["contracts/id", "contracts/status"]},
        {"all": ["awards/id", "awards/status"]},
    ]
}

_AWARD_CONTRACT_VALUE = {
    "any": [
        {"all": ["contracts/id", "contracts/status", "contracts/value/amount", "contracts/value/currency"]},
        {"all": ["awards/id", "awards/status", "awards/value/amount", "awards/value/currency"]},
    ]
}

_BIDDERS_COUNT = {"any": ["tender/numberOfTenderers", "tender/tenderers/id", "bids/details/tenderers/id"]}

_AWARD_CONTRACT_ITEMS_VALUE = {
    "any": [
        {
            "all": [
                "awards/status",
                "awards/value/amount",
                "awards/value/currency",
                "awards/items/classification/id",
                "awards/items/classification/scheme",
            ]
        },
        {
            "all": [
                "contracts/status",
                "contracts/value/amount",
                "contracts/value/currency",
                "contracts/items/classification/id",
                "contracts/items/classification/scheme",
            ]
        },
    ]
}

_AWARD_CONTRACT_STATUS_ONLY = {"any": ["awards/status", "contracts/status"]}

_CONTRACT_START_DATE = {"any": ["contracts/period/startDate", "awards/contractPeriod/startDate"]}

_PLANNING = {
    "any": [
        {"all": ["planning/budget/amount/amount", "planning/budget/amount/currency"]},
        {"all": ["tender/value/amount", "tender/value/currency"]},
    ]
}

INDICATORS_REQUIREMENTS = {
    "Total number of procedures": {
        "id": ["U001"],
        "rule": {"all": ["ocid"]},
    },
    "Total number of procuring entities": {
        "id": ["U002"],
        "rule": {"all": ["ocid", _BUYER]},
    },
    "Total number of unique bidders": {
        "id": ["U003"],
        "rule": {"all": ["ocid", _BIDDERS]},
    },
    "Total number of awarded suppliers": {
        "id": ["U004"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/status"]},
    },
    "Total number of procedures by year or month": {
        "id": ["U005"],
        "rule": {"all": ["ocid", "date"]},
    },
    "Total value awarded": {
        "id": ["U006"],
        "rule": {"all": ["ocid", "awards/status", "awards/value/amount", "awards/value/currency"]},
    },
    "Share of procedures by status": {
        "id": ["U007"],
        "rule": {"all": ["ocid", "tender/status"]},
    },
    "Number of procedures by item type": {
        "id": ["U008"],
        "rule": {"all": ["ocid", _ITEMS]},
    },
    "Proportion of procedures by procurement category": {
        "id": ["U009"],
        "rule": {"all": ["ocid", "tender/mainProcurementCategory"]},
    },
    "Percent of tenders by procedure type": {
        "id": ["U010"],
        "rule": {"all": ["ocid", "tender/procurementMethod"]},
    },
    "Percent of tenders awarded by means of competitive procedures": {
        "id": ["U011"],
        "rule": {"all": ["ocid", "tender/procurementMethod", "awards/status"]},
    },
    "Percent of contracts awarded under each procedure type": {
        "id": ["U012"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _AWARD_CONTRACT_STATUS]},
    },
    "Total contracted value awarded under each procedure type": {
        "id": ["U013"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _AWARD_CONTRACT_VALUE]},
    },
    "Total awarded value of tenders awarded by means of competitive procedures": {
        "id": ["U014"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _AWARD_CONTRACT_VALUE]},
    },
    "Proportion of single bid tenders": {
        "id": ["U015"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _BIDDERS_COUNT]},
    },
    "Proportion of value awarded in single bid tenders vs competitive tenders": {
        "id": ["U016"],
        "rule": {
            "all": [
                "ocid",
                "tender/procurementMethod",
                "awards/status",
                "awards/value/amount",
                "awards/value/currency",
                _BIDDERS_COUNT,
            ]
        },
    },
    "Mean number of bidders per tender": {
        "id": ["U017"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _BIDDERS_COUNT]},
    },
    "Median number of bidders per tender": {
        "id": ["U018"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _BIDDERS_COUNT]},
    },
    "Mean number of bidders by item type": {
        "id": ["U019"],
        "rule": {"all": ["ocid", "tender/procurementMethod", _BIDDERS_COUNT, _ITEMS]},
    },
    "Number of suppliers by item type": {
        "id": ["U020"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", _ITEMS]},
    },
    "Number of new bidders in a system ": {
        "id": ["U021"],
        "rule": {"all": ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"]},
    },
    "Percent of new bidders to all bidders ": {
        "id": ["U022"],
        "rule": {"all": ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"]},
    },
    "Percent of tenders with at least three participants deemed qualified": {
        "id": ["U023"],
        "rule": {"all": ["ocid", "bids/details/tenderers/id", "bids/details/id", "bids/details/status"]},
    },
    "Mean percent of bids which are disqualified": {
        "id": ["U024"],
        "rule": {"all": ["tender/id", "bids/details/id", "bids/details/status"]},
    },
    "Percent of contracts awarded to top 10 suppliers with largest contracted totals": {
        "id": ["U025"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", _AWARD_CONTRACT_VALUE]},
    },
    "Mean number of unique suppliers per buyer": {
        "id": ["U026"],
        "rule": {"all": ["ocid", "awards/suppliers/id", "awards/suppliers/name", _BUYER]},
    },
    "Number of new awarded suppliers ": {
        "id": ["U027"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"]},
    },
    "Percent of awards awarded to new suppliers": {
        "id": ["U028"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"]},
    },
    "Total awarded value awarded to new suppliers": {
        "id": ["U029"],
        "rule": {
            "all": [
                "awards/id",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/date",
                "awards/value/amount",
                "awards/value/currency",
            ]
        },
    },
    "Percent of new suppliers to all suppliers": {
        "id": ["U030"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"]},
    },
    "Percent of growth of new awarded suppliers in a system": {
        "id": ["U031"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"]},
    },
    "Percent of total awarded value awarded to recurring suppliers": {
        "id": ["U032"],
        "rule": {
            "all": [
                "awards/id",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/date",
                "awards/value/amount",
                "awards/value/currency",
            ]
        },
    },
    "Mean number of bids necessary to win": {
        "id": ["U033"],
        "rule": {"all": ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"]},
    },
    "Market concentration, market share of the largest company in the market": {
        "id": ["U034"],
        "rule": {"all": ["awards/suppliers/id", "awards/suppliers/name", _AWARD_CONTRACT_ITEMS_VALUE]},
    },
    "Proportion of contracts awarded by supplier by non competitive procedures": {
        "id": ["U035"],
        "rule": {
            "all": ["ocid", "tender/procurementMethod", "awards/status", "awards/suppliers/id", "awards/suppliers/name"]
        },
    },
    "Region of the supplier": {
        "id": ["U036"],
        "rule": {"all": ["parties/roles", "parties/identifier/id", "parties/address/region"]},
    },
    "Number of bids submitted by supplier": {
        "id": ["U037"],
        "rule": {"all": ["awards/suppliers/id", _BIDDERS]},
    },
    "Success rate of bidders": {
        "id": ["U038"],
        "rule": {"all": ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"]},
    },
    "Number of unique items classifications awarded by supplier": {
        "id": ["U039"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", _ITEMS]},
    },
    "Total value awarded by supplier": {
        "id": ["U040"],
        "rule": {"all": ["awards/suppliers/id", "awards/suppliers/name", _AWARD_CONTRACT_VALUE]},
    },
    "Share of total value awarded by supplier": {
        "id": ["U041"],
        "rule": {"all": ["awards/suppliers/id", "awards/suppliers/name", _AWARD_CONTRACT_VALUE]},
    },
    "Total number of contracts awarded by supplier": {
        "id": ["U042"],
        "rule": {"all": ["awards/id", "awards/suppliers/id", "awards/suppliers/name", _AWARD_CONTRACT_STATUS_ONLY]},
    },
    "Number of procuring entities by supplier": {
        "id": ["U043"],
        "rule": {"all": ["ocid", "awards/suppliers/id", "awards/suppliers/name", _BUYER]},
    },
    "Share of single bid awards by supplier": {
        "id": ["U044"],
        "rule": {
            "all": [
                "ocid",
                "awards/suppliers/id",
                "awards/suppliers/name",
                "awards/status",
                "tender/procurementMethod",
                _BIDDERS_COUNT,
            ]
        },
    },
    "Percent of tenders with linked procurement plans": {
        "id": ["U045"],
        "rule": {"all": ["tender/id", "tender/documents/documentType"]},
    },
    "Percent of contracts which publish information on debarments": {
        "id": ["U046"],
        "rule": {"all": ["contracts/id", "contracts/implementation/documents/documentType"]},
    },
    "The percent of tenders for which the tender documentation was added after publication of the announcement ": {
        "id": ["U047"],
        "rule": {
            "all": [
                "tender/id",
                "tender/documents/documentType",
                "tender/documents/documentType",
                "tender/documents/datePublished",
            ]
        },
    },
    "Mean number of contract amendments per buyer": {
        "id": ["U048"],
        "rule": {"all": ["ocid", "contracts/id", "contracts/amendments", _BUYER]},
    },
    "Percent of tenders which have been closed for more than 30 days, but whose basic awards information is not published": {
        "id": ["U049"],
        "rule": {
            "all": [
                "tender/id",
                "tender/tenderPeriod/endDate",
                "awards/id",
                "awards/date",
                "awards/status",
                "awards/value/amount",
                "awards/suppliers/id",
                "awards/suppliers/name",
            ]
        },
    },
    "Percent of awards which are older than 30 days, but whose contract is not published": {
        "id": ["U050"],
        "rule": {
            "all": [
                "awards/id",
                "awards/date",
                "contracts/awardID",
                "contracts/status",
                "contracts/dateSigned",
                "contracts/documents/documentType",
            ]
        },
    },
    "Percent of tenders that do not specify place of delivery": {
        "id": ["U051"],
        "rule": {"all": ["ocid", "tender/items/deliveryLocation", "tender/items/deliveryAddress"]},
    },
    "Percent of tenders that do not specify date of delivery": {
        "id": ["U052"],
        "rule": {
            "all": [
                "tender/milestones/id",
                "tender/milestones/type",
                "tender/milestones/description",
                "tender/milestones/dueDate",
            ]
        },
    },
    "Percent of tenders with short titles for example fewer than 10 characters in the title": {
        "id": ["U053"],
        "rule": {"all": ["tender/id", "tender/title"]},
    },
    "Percent of tenders with short descriptions for instance fewer than 30 characters in the description": {
        "id": ["U054"],
        "rule": {"all": ["tender/id", "tender/description"]},
    },
    "Percent of tenders that do not include detailed item codes or item descriptions": {
        "id": ["U055"],
        "rule": {"all": ["tender/id", "tender/items/classification/id", "tender/items/classification/scheme"]},
    },
    "Percent of contracts that do not have amendments": {
        "id": ["U056"],
        "rule": {"all": ["contracts/id", "contracts/amendments"]},
    },
    "Percent of contracts which publish contract implementation details financial": {
        "id": ["U057"],
        "rule": {
            "all": [
                "contracts/implementation/transactions/id",
                "contracts/implementation/transactions/value/amount",
                "contracts/implementation/transactions/value/currency",
            ]
        },
    },
    "Percent of contracts which publish contract implementation details physical": {
        "id": ["U058"],
        "rule": {
            "all": [
                "contracts/implementation/milestones/type",
                "contracts/implementation/milestones/id",
                "contracts/implementation/milestones/dueDate",
                "contracts/implementation/milestones/status",
            ]
        },
    },
    "Average duration of tendering period days": {
        "id": ["U059"],
        "rule": {"all": ["ocid", "tender/tenderPeriod/startDate", "tender/tenderPeriod/endDate"]},
    },
    "Average duration of decision period days": {
        "id": ["U060"],
        "rule": {"all": ["ocid", "tender/tenderPeriod/endDate", "awards/date"]},
    },
    "Average days from award date to start of implementation": {
        "id": ["U061"],
        "rule": {"all": ["awards/id", "awards/date", _CONTRACT_START_DATE]},
    },
    "Days between award date and tender start date": {
        "id": ["U062"],
        "rule": {"all": ["ocid", "tender/tenderPeriod/startDate", "awards/date"]},
    },
    "Percent of canceled tenders to awarded tenders": {
        "id": ["U063"],
        "rule": {"all": ["ocid", "tender/status", "awards/status"]},
    },
    "Percent of contracts which are canceled": {
        "id": ["U064"],
        "rule": {"all": ["contracts/id", "contracts/status"]},
    },
    "Price variation of same item across all awards": {
        "id": ["U065"],
        "rule": {
            "any": [
                {
                    "all": [
                        "awards/status",
                        "awards/value/amount",
                        "awards/value/currency",
                        "awards/items/classification/id",
                        "awards/items/classification/scheme",
                        "awards/items/quantity",
                        "awards/items/unit",
                    ]
                },
                {
                    "all": [
                        "contracts/status",
                        "contracts/value/amount",
                        "contracts/value/currency",
                        "contracts/items/classification/id",
                        "contracts/items/classification/scheme",
                        "contracts/items/quantity",
                        "contracts/items/unit",
                    ]
                },
            ]
        },
    },
    "Percent of contracts that exceed budget": {
        "id": ["U066"],
        "rule": {
            "all": [
                "ocid",
                "contracts/status",
                "contracts/value/amount",
                "contracts/value/currency",
                _PLANNING,
            ]
        },
    },
    "Mean percent overrun of contracts that exceed budget": {
        "id": ["U067"],
        "rule": {
            "all": [
                "ocid",
                "contracts/status",
                "contracts/value/amount",
                "contracts/value/currency",
                _PLANNING,
            ]
        },
    },
    "Total percent savings difference between budget and contract value": {
        "id": ["U068"],
        "rule": {
            "all": [
                "ocid",
                "planning/budget/amount/amount",
                "planning/budget/amount/currency",
                "contracts/value/amount",
                "contracts/value/currency",
            ]
        },
    },
    "Total percent savings difference between tender value estimate and contract value": {
        "id": ["U069"],
        "rule": {
            "all": [
                "ocid",
                "tender/value/amount",
                "tender/value/currency",
                "contracts/value/amount",
                "contracts/value/currency",
            ]
        },
    },
    "Percent of contracts completed on time ": {
        "id": ["U070"],
        "rule": {"all": ["contracts/id", "contracts/period/endDate", "contracts/status"]},
    },
    "Share of contracts whose milestones are completed on time": {
        "id": ["U071"],
        "rule": {
            "all": [
                "contracts/id",
                "contracts/implementation/milestones/dueDate",
                "contracts/implementation/milestones/dateMet",
            ]
        },
    },
}


def check_rule(rule, fields_list):
    """Check if a DSL rule is satisfied against available fields.

    Returns a tuple of (satisfied, missing_fields) where satisfied is a boolean
    and missing_fields is a list of fields that are missing.
    """
    if isinstance(rule, str):
        satisfied = rule in fields_list
        return satisfied, [] if satisfied else [rule]
    if "all" in rule:
        missing = []
        for sub in rule["all"]:
            ok, sub_missing = check_rule(sub, fields_list)
            if not ok:
                missing.extend(sub_missing)
        return len(missing) == 0, missing
    if "any" in rule:
        all_missings = []
        for sub in rule["any"]:
            ok, sub_missing = check_rule(sub, fields_list)
            if ok:
                return True, []
            all_missings.append(sub_missing)
        best = min(all_missings, key=len) if all_missings else []
        return False, best


def get_all_fields(rule):
    """Get all field paths referenced in a DSL rule."""
    if isinstance(rule, str):
        return [rule]
    if "all" in rule:
        fields = []
        for sub in rule["all"]:
            fields.extend(get_all_fields(sub))
        return fields
    if "any" in rule:
        fields = []
        for sub in rule["any"]:
            fields.extend(get_all_fields(sub))
        return fields
    return []


def usability_checks(fields_list, indicators_requirements):
    """
    Return a table of the usability checks.

    It indicates if the fields needed to calculate a particular indicator are present.
    """
    results_list = []
    missing_fields = []
    fields_needed = []

    for indicator_data in indicators_requirements.values():
        ok, missing = check_rule(indicator_data["rule"], fields_list)
        result = "possible to calculate" if ok else "missing fields"
        results_list.append(result)
        missing_fields.append(missing)
        fields_needed.append(", ".join(get_all_fields(indicator_data["rule"])))

    # Generate dataframe

    indicatordf = pd.DataFrame(
        list(
            zip(
                list(indicators_requirements),
                [indicators_requirements[i]["id"] for i in indicators_requirements],
                fields_needed,
                strict=True,
            )
        ),
        columns=["indicator", "U_id", "fields needed"],
    )
    indicatordf["U_id"] = indicatordf["U_id"].apply(lambda x: ", ".join(map(str, x)))
    indicatordf["calculation"] = results_list
    indicatordf["missing fields"] = missing_fields
    indicatordf["missing fields"] = indicatordf["missing fields"].apply(lambda x: ", ".join(map(str, x)))

    return indicatordf


def check_usability_indicators(lang, result):

    gc = authenticate_gspread()

    if lang.value == "English":
        # Use case guide: Indicators linked to OCDS #public
        spreadsheet_key = "1j-Y0ktZiOyhZzi-2GSabBCnzx6fF5lv8h1KYwi_Q9GM"
    else:
        # [ES] of Use case guide: Indicators linked to OCDS #public
        spreadsheet_key = "1l_p_e1iNUUuR5AObTJ8EY9VrcCLTAq3dnG_Fj73UH9w"

    worksheet = gc.open_by_key(spreadsheet_key).sheet1

    # get_all_values gives a list of rows.
    rows = worksheet.get_all_values()
    # Convert to a DataFrame and render.

    indicators = pd.DataFrame(rows)
    indicators = indicators.rename(columns=indicators.iloc[0]).drop(indicators.index[0])

    if lang.value == "English":
        indicatorsdf = indicators.iloc[:, [0, 3, 4, 9]]
        result_final = result.merge(indicatorsdf, on="U_id")
    else:
        indicatorsdf = indicators.iloc[:, [0, 3, 4, 5, 9]]
        result_final = indicatorsdf.merge(result, on="U_id").drop(["indicator"], axis=1)
        result_final = result_final.rename(
            columns={
                "fields needed": "Campos necesarios",
                "calculation": "¿Se puede calcular?",
                "missing fields": "Campos faltantes",
                "coverage": "Cobertura",
            },
        )
        result_final = result_final.replace(
            {"¿Se puede calcular?": {"possible to calculate": "sí", "missing fields": "campos faltantes"}}
        )
    return result_final


def is_relevant(field_list):
    final_result = []
    for key, value in RELEVANT_RULES.items():
        rule_result = {"rule": key, "possible_to_calculate": "No", "available_fields": [], "missing_fields": []}
        for field in value:
            if isinstance(field, str):
                if field in field_list:
                    rule_result["possible_to_calculate"] = "Yes"
                    rule_result["available_fields"].append(field)
                else:
                    rule_result["missing_fields"].append(field)
            else:
                missing = [item for item in field if item not in field_list]
                rule_result["available_fields"].extend(item for item in field if item in field_list)
                rule_result["missing_fields"].extend(missing)
                if not missing:
                    rule_result["possible_to_calculate"] = "Yes"
        final_result.append(rule_result)

    final_result = pd.DataFrame(final_result)
    relevant = (final_result["possible_to_calculate"] == "Yes").all()
    return relevant, final_result


def get_coverage(indicators_dic):
    coverage = []
    for data in indicators_dic.values():
        fields = get_all_fields(data["rule"])
        result = calculate_coverage(fields, "release_summary")
        result_value = pd.to_numeric(result["total_percentage"][0])
        coverage.append(result_value)
    return coverage


def most_common_fields_to_calculate_indicators(indicators_dict, fields_table):
    flat_list = []
    for data in indicators_dict.values():
        flat_list.extend(get_all_fields(data["rule"]))
    fields_list = Counter(flat_list)

    fields_count = (
        pd.DataFrame.from_dict(fields_list, orient="index")
        .reset_index()
        .rename(columns={"index": "field", 0: "number of indicators"})
    )

    fields_count = fields_count.sort_values("number of indicators", ascending=False).reset_index(drop=True)
    fields_count["published"] = np.where(fields_count["field"].isin(fields_table["path"]), "yes", "no")

    return fields_count


def get_usability_language_select_box():
    style = {"description_width": "initial"}
    languages = ["Spanish", "English"]
    return widgets.Dropdown(options=languages, description="language", style=style)
